In [17]:
# Required only for colab environments + GPU
!pip install jax==0.3.25 jaxlib==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# Install required libraries
!pip install -q dalle-mini
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.3.25.tar.gz (1.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


ERROR: Ignored the following yanked versions: 0.4.32
ERROR: Could not find a version that satisfies the requirement jaxlib==0.3.25 (from versions: 0.4.17, 0.4.18, 0.4.19, 0.4.20, 0.4.21, 0.4.22, 0.4.23, 0.4.25, 0.4.26, 0.4.27, 0.4.28, 0.4.29, 0.4.30, 0.4.31, 0.4.33, 0.4.34, 0.4.35, 0.4.36)
ERROR: No matching distribution found for jaxlib==0.3.25


In [11]:
choose_model = 'DALL-E Mega Full'

#option for image genration models
models = {'DALL-E Mini':'dalle-mini/dalle-mini/mini-1:v0',
          'DALL-E MEga':'dalle-mini/dalle-mini/mega-1-fp16:lateset',
          'DALL-E Mega Full':'dalle-mini/dalle-mini/mega-1:latest'}


DALLE_COMMIT_ID = None

DALLE_MODEL = models[choose_model]


# VQGAN model
VQGAN_REPO = "dalle-mini/vqgan_imagenet_f16_16384"
VQGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"

#Loading models and tokenizer
from dalle_mini import DalleBart, DalleBartProcessor
from vqgan_jax.modeling_flax_vqgan import VQModel
from transformers import CLIPProcessor, FlaxCLIPModel

#Loading dalle mini

model,params = DalleBart.from_pretrained(DALLE_MODEL, revision = DALLE_COMMIT_ID, dtype = jnp.float16, _do_init=False)

#Loading VQGAN
vqgan, vqgan_params = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID,_do_init=False)

from flax.jax_utils import replicate

params = replicate(params)
vqgan_params = replicate(vqgan_params)

from functools import partial

#model inference
@partial(jax.pmap, axis_name='batch', static_broadcasted_argnums=(3,4,5,6))
def p_generate(tokenized_prompt,key,params,top_k,top_p,temperature,condition_scale):
  return model.generate(**tokenized_prompt, prng_key=key,params= params, top_k=top_k,top_p=top_p,tempreture =  temperature, condition_scale=condition_scale)

#decoder image
@partial(jax.pmap, axis_name='batch')

def p_decode(indices,params):
  return vqgan.decode_code(indices,params=params)

from dalle_mini import DalleBartProcessor

processor = DalleBartProcessor.from_pretrained(DALLE_MODEL,revision = DALLE_COMMIT_ID)

AttributeError: module 'jax.numpy' has no attribute 'DeviceArray'

In [12]:
# Required libraries
import jax
import jax.numpy as jnp
from flax.jax_utils import replicate
from transformers import CLIPProcessor, FlaxCLIPModel
from dalle_mini import DalleBart, DalleBartProcessor
from vqgan_jax.modeling_flax_vqgan import VQModel
from functools import partial

# Select the DALL-E model
choose_model = "DALL-E Mega Full"

# Model options
models = {
    "DALL-E Mini": "dalle-mini/dalle-mini/mini-1:v0",
    "DALL-E Mega": "dalle-mini/dalle-mini/mega-1-fp16:latest",
    "DALL-E Mega Full": "dalle-mini/dalle-mini/mega-1:latest"
}

DALLE_COMMIT_ID = None  # Default commit ID
DALLE_MODEL = models[choose_model]

# VQGAN model repository details
VQGAN_REPO = "dalle-mini/vqgan_imagenet_f16_16384"
VQGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"

# Load DALL-E model and parameters
model, params = DalleBart.from_pretrained(
    DALLE_MODEL,
    revision=DALLE_COMMIT_ID,
    dtype=jnp.float16,
    _do_init=False
)

# Load VQGAN model and parameters
vqgan, vqgan_params = VQModel.from_pretrained(
    VQGAN_REPO,
    revision=VQGAN_COMMIT_ID,
    _do_init=False
)

# Replicate parameters for JAX parallel computation
params = replicate(params)
vqgan_params = replicate(vqgan_params)

# Function for model inference
@partial(jax.pmap, axis_name="batch", static_broadcasted_argnums=(3, 4, 5, 6))
def p_generate(tokenized_prompt, key, params, top_k, top_p, temperature, condition_scale):
    return model.generate(
        **tokenized_prompt,
        prng_key=key,
        params=params,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        condition_scale=condition_scale,
    )

# Function for decoding image indices to actual images
@partial(jax.pmap, axis_name="batch")
def p_decode(indices, params):
    return vqgan.decode_code(indices, params=params)

# Load DalleBartProcessor
processor = DalleBartProcessor.from_pretrained(
    DALLE_MODEL,
    revision=DALLE_COMMIT_ID
)

print("Models and processor loaded successfully!")


AttributeError: module 'jax.numpy' has no attribute 'DeviceArray'

In [16]:
# Install necessary libraries if not already installed
# pip install dalle-mini flax jax jaxlib transformers vqgan-jax

import jax
import jax.numpy as jnp
import DalleBart, DalleBartProcessor
from flax.jax_utils import replicate
from transformers import FlaxCLIPModel
from vqgan_jax.modeling_flax_vqgan import VQModel
from functools import partial
import numpy as np
import matplotlib.pyplot as plt

# Load DALL-E and VQGAN model details
DALLE_MODEL = "dalle-mini/dalle-mini/mega-1:latest"
DALLE_COMMIT_ID = None

VQGAN_REPO = "dalle-mini/vqgan_imagenet_f16_16384"
VQGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"

# Load DALL-E model
model, params = DalleBart.from_pretrained(
    DALLE_MODEL, revision=DALLE_COMMIT_ID, dtype=jnp.float16, _do_init=False
)

# Load VQGAN model
vqgan, vqgan_params = VQModel.from_pretrained(
    VQGAN_REPO, revision=VQGAN_COMMIT_ID, _do_init=False
)

# Replicate parameters for multi-device use
params = replicate(params)
vqgan_params = replicate(vqgan_params)

# Model inference function
@partial(jax.pmap, axis_name="batch", static_broadcasted_argnums=(3, 4, 5, 6))
def p_generate(tokenized_prompt, key, params, top_k, top_p, temperature, condition_scale):
    return model.generate(
        **tokenized_prompt,
        prng_key=key,
        params=params,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        condition_scale=condition_scale,
    )

# Decode function
@partial(jax.pmap, axis_name="batch")
def p_decode(indices, params):
    return vqgan.decode_code(indices, params=params)

# Load the processor
processor = DalleBartProcessor.from_pretrained(DALLE_MODEL, revision=DALLE_COMMIT_ID)

# Generate an image from a prompt
def generate_image(prompt, num_images=1, seed=0, top_k=50, top_p=0.9, temperature=1.0, condition_scale=10.0):
    # Tokenize prompt
    tokenized_prompt = processor([prompt])
    tokenized_prompt = replicate(tokenized_prompt)

    # Prepare random seed
    key = jax.random.PRNGKey(seed)
    key = jax.random.split(key, jax.local_device_count())

    # Generate image tokens
    generated_tokens = p_generate(
        tokenized_prompt,
        key,
        params,
        top_k,
        top_p,
        temperature,
        condition_scale,
    )

    # Extract sequences and decode
    decoded_images = p_decode(generated_tokens.sequences[:, :, 1:], vqgan_params)
    return decoded_images

# Visualize the image
def visualize_image(image):
    image = np.asarray(image).squeeze()
    plt.imshow(image)
    plt.axis("off")
    plt.show()

# Example usage
if __name__ == "__main__":
    prompt = "a futuristic cityscape at sunset"
    images = generate_image(prompt, num_images=1)

    # Visualize the first generated image
    visualize_image(images[0])


ModuleNotFoundError: No module named 'DalleBart'